In [1]:
import keras
import tensorflow
from keras import Layer
from keras.layers import Dense # type: ignore
import keras_tuner

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.3.0) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
def build_model(hp):
    model = keras.Sequential()
    
    model.add(Dense(units = hp.Int('units', min_value = 32, max_value = 512, step = 32), activation = 'relu'))
    
    model.add(Dense(10, activation = 'softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [3]:
build_model(keras_tuner.HyperParameters())

<Sequential name=sequential, built=False>

In [ ]:
def build_model2(hp):
    model = keras.Sequential()
    
    model.add(Dense(units = hp.Int('units', min_value = 32, max_value = 512, step = 32), activation = 'relu'), activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid']))
    
    if hp.Boolean('use_dropout'):
        model.add(Layer.Dropout(rate = hp.Choice('dropout_rate', values = [0.2, 0.3, 0.4, 0.5])))
    model.add(Dense(10, activation = 'softmax'))
    
    learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4], sampling = 'log')
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model